In [1]:
import os

In [2]:
%pwd

'h:\\Projects\\MLOps\\Chicken-Disease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'h:\\Projects\\MLOps\\Chicken-Disease-Classification-Project'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbackConfig:

        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

h:\Projects\MLOps\Chicken-Disease-Classification-Project\chicken\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
h:\Projects\MLOps\Chicken-Disease-Classification-Project\chicken\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
h:\Projects\MLOps\Chicken-Disease-Classification-Project\chicken\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.3

In [14]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config


    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, f"run-{timestamp}",
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property

    def _create_ckpt_callback(self):

        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callback,
            self._create_ckpt_callback
        ]

In [15]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallback(config=prepare_callback_config)
    callbacks_list = prepare_callback.get_tb_ckpt_callbacks()

except Exception as e:
    raise e



[2025-08-12 13:09:45,158: INFO: common: Created directory at: artifacts]
[2025-08-12 13:09:45,179: INFO: common: Created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-08-12 13:09:45,183: INFO: common: Created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [16]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
print(f"Current timestamp: {timestamp}")

Current timestamp: 2025-08-12-13-48-18
